In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical

In [2]:
# Load dataset
data = pd.read_csv("seattle-weather.csv")

In [3]:
# Encode categorical data
label_encoder = LabelEncoder()
data['weather'] = label_encoder.fit_transform(data['weather'])

In [4]:
# Normalize features
scaler = MinMaxScaler()
data[['precipitation', 'temp_max', 'temp_min', 'wind']] = scaler.fit_transform(data[['precipitation', 'temp_max', 'temp_min', 'wind']])

In [5]:
# Split data into features and target
X = data[['precipitation', 'temp_max', 'temp_min', 'wind']]
y = data['weather']

In [6]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

In [18]:
# Convert target to one-hot encoding
y = to_categorical(y)

In [19]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Reshape input data for RNN
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [42]:
from tensorflow.keras.layers import Bidirectional, Dropout

In [44]:
# Build RNN model with more layers and dropout
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [45]:
# Train the model with adjusted parameters
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
19/19 [==============================] - 21s 246ms/step - loss: 1.5455 - accuracy: 0.5548 - val_loss: 1.4497 - val_accuracy: 0.6860
Epoch 2/100
19/19 [==============================] - 0s 17ms/step - loss: 1.3080 - accuracy: 0.6002 - val_loss: 1.1542 - val_accuracy: 0.6587
Epoch 3/100
19/19 [==============================] - 0s 18ms/step - loss: 1.0814 - accuracy: 0.6370 - val_loss: 1.1039 - val_accuracy: 0.6724
Epoch 4/100
19/19 [==============================] - 0s 17ms/step - loss: 1.0567 - accuracy: 0.6490 - val_loss: 1.0687 - val_accuracy: 0.6007
Epoch 5/100
19/19 [==============================] - 0s 19ms/step - loss: 1.0255 - accuracy: 0.6344 - val_loss: 1.0324 - val_accuracy: 0.6792
Epoch 6/100
19/19 [==============================] - 0s 17ms/step - loss: 0.9800 - accuracy: 0.6738 - val_loss: 0.9821 - val_accuracy: 0.6860
Epoch 7/100
19/19 [==============================] - 0s 16ms/step - loss: 0.9341 - accuracy: 0.6781 - val_loss: 0.9348 - val_accuracy: 0.6792
Epoc

In [46]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy:', accuracy)



Accuracy: 0.8156996369361877


In [47]:
# Make predictions on new data
new_data = np.array([[20.0, 0.0, -5.0, 4.7]])  # Example new data
new_data = scaler.transform(new_data)
new_data = new_data.reshape((1, 1, new_data.shape[1]))
prediction = model.predict(new_data)
predicted_weather = label_encoder.inverse_transform([np.argmax(prediction)])
print('Predicted weather:', predicted_weather)

c:\Users\A S U S\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 4s 4s/step
Predicted weather: ['snow']


Model Pertama:

Model pertama menggunakan satu lapisan LSTM dengan 50 neuron, diikuti oleh lapisan Dense dengan 4 neuron untuk output.
Fungsi aktivasi yang digunakan adalah ReLU untuk lapisan LSTM dan softmax untuk lapisan output.
Fungsi kerugian yang digunakan adalah categorical_crossentropy karena kita memiliki masalah klasifikasi multikelas.
Model pertama menggunakan metode optimasi Adam dengan default learning rate.
Model Kedua:

Model kedua menggunakan dua lapisan LSTM, masing-masing dengan 100 dan 50 neuron secara berturut-turut. Lapisan LSTM pertama menggunakan bidirectional LSTM untuk memperluas pemahaman temporal data.
Kami menambahkan lapisan dropout dengan dropout rate 0.2 setelah lapisan LSTM pertama untuk mengurangi overfitting.
Fungsi aktivasi yang sama digunakan untuk lapisan LSTM (ReLU) dan lapisan output (softmax).
Model kedua menggunakan metode optimasi yang sama (Adam) dengan learning rate default.
Perbedaan utama yang menyebabkan peningkatan akurasi dalam model kedua adalah:

Kompleksitas Model: Model kedua memiliki struktur yang lebih kompleks dengan dua lapisan LSTM dan dropout layer, yang memungkinkan model untuk mempelajari representasi yang lebih baik dari data.

Bidirectional LSTM: Penggunaan bidirectional LSTM pada lapisan pertama memungkinkan model untuk memahami konteks temporal dalam kedua arah, yang dapat membantu dalam memodelkan pola yang lebih kompleks dalam data.

Regularisasi: Penambahan dropout layer setelah lapisan LSTM pertama membantu mengurangi overfitting dengan menghapus beberapa koneksi antar-neuron secara acak selama pelatihan.

Jumlah Neuron: Penambahan jumlah neuron pada lapisan LSTM kedua juga dapat meningkatkan kapasitas model, yang mungkin diperlukan untuk menangkap pola yang lebih kompleks dalam data.